In [138]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
import re
import gensim

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CapoBizkitz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CapoBizkitz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\CapoBizkitz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [139]:
train = pd.read_csv('https://raw.githubusercontent.com/Tariq60/LIAR-PLUS/master/dataset/tsv/train2.tsv', delimiter='\t', encoding='utf-8', names=
                        ["json ID", "label", "statement", "subject", "speaker", "job title", "state", "party",
                         "barely true", "false", "half true", "mostly true", "pants on fire", "los", "justification"])
train.name = 'Training Data'

test = pd.read_csv('https://raw.githubusercontent.com/Tariq60/LIAR-PLUS/master/dataset/tsv/test2.tsv', delimiter='\t', encoding='utf-8', names=
                        ["json ID", "label", "statement", "subject", "speaker", "job title", "state", "party",
                         "barely true", "false", "half true", "mostly true", "pants on fire", "los", "justification"])
test.name = 'Testing Data'

In [140]:
null_columns=train.columns[train.isnull().any()]
train[null_columns].isnull().sum()

json ID             2
label               2
statement           2
subject             4
speaker             4
job title        2899
state            2210
party               4
barely true         4
false               4
half true           4
mostly true         4
pants on fire       4
los               104
justification      88
dtype: int64

In [141]:
null_columns=test.columns[test.isnull().any()]
test[null_columns].isnull().sum()

job title        325
state            262
los               17
justification      9
dtype: int64

In [142]:
print(train[train["label"].isnull()][null_columns])

    job title state  los justification
NaN       NaN   NaN  NaN           NaN
NaN       NaN   NaN  NaN           NaN


In [143]:
train = train[train['json ID'].notna()]

In [144]:
print(train[train["label"].isnull()][null_columns])

Empty DataFrame
Columns: [job title, state, los, justification]
Index: []


In [145]:
train.shape

(10240, 15)

In [146]:
test.shape

(1267, 15)

In [147]:
train['label'].value_counts()

half-true      2114
false          1995
mostly-true    1962
true           1676
barely-true    1654
pants-fire      839
Name: label, dtype: int64

In [148]:
train['new'] = train['statement'].map(str) + train['justification'].map(str)
test['new'] = test['statement'].map(str) + test['justification'].map(str)

In [149]:
ppr_train = pd.read_csv("../../dataset/PreprocessedLiar/train.csv")

In [150]:
ppr_test = pd.read_csv("../../dataset/PreprocessedLiar/test.csv")

In [151]:
dataset = ppr_train.append(ppr_test, ignore_index=True)

In [152]:
ppr_dataset = pd.DataFrame(dataset['cleaned_statement'].tolist()).apply(lambda x: x.str.len()).mean(axis=1)

In [153]:
avg_ppr_words_dataset = ppr_dataset.mean()
print(avg_ppr_words_dataset)

318.7786564699748


In [154]:
dataset['word_count'] = dataset['cleaned_statement'].str.count(' ') + 1

In [155]:
max_value = dataset['word_count'].max()
print(max_value)

753


In [156]:
df_mask=dataset['word_count']<=50
filtered_df = dataset[df_mask]
print(filtered_df)

       articleId                                  cleaned_statement  \
0           2635  say annies list political group support trimes...   
2            324  hillary clinton agrees john mccain voting geor...   
4           9028  economic turnaround started term crist said ec...   
5          12465  chicago bear starting quarterback year total n...   
6           2342  dunnam lived district represents year determin...   
...          ...                                                ...   
11501       6699  menendez voted enact sale home santorum correc...   
11502       7334  say budget provides highest state funding leve...   
11504      10710  early edward kennedy secretly offered help sov...   
11505       3186  say permit languished strickland director day ...   
11506       6743  say governor going state talking fund income b...   

             label  word_count  
0            false          28  
2      mostly-true          39  
4        half-true          50  
5             t

In [161]:
filtered_df.sample(10)

,articleId,cleaned_statement,label
7762,10530,student attempt suicide year college campus ma...,half-true
9144,7546,hole earth thats going proposed gogebic bigger...,true
8056,4505,baby boomer retiring simply shred proof public...,true
9891,12764,say debbie wasserman schultz voted huge subsid...,mostly-true
3903,274,building cayman island supposedly house corpor...,half-true
9674,7897,coal industrybut decision based state funding ...,false
7790,12032,million american work example obama think pres...,half-true
1322,8252,kenosha casino state largest employer state co...,mostly-true
5214,1940,spends billion research development energy spe...,half-true
5968,1818,history department university texas registered...,false


In [158]:
filtered_df.drop("word_count", axis=1, inplace=True)

c:\Users\CapoBizkitz\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [159]:
filtered_df

,articleId,cleaned_statement,label
0,2635,say annies list political group support trimes...,false
2,324,hillary clinton agrees john mccain voting geor...,mostly-true
4,9028,economic turnaround started term crist said ec...,half-true
5,12465,chicago bear starting quarterback year total n...,true
6,2342,dunnam lived district represents year determin...,barely-true
...,...,...,...
11501,6699,menendez voted enact sale home santorum correc...,pants-fire
11502,7334,say budget provides highest state funding leve...,half-true
11504,10710,early edward kennedy secretly offered help sov...,barely-true
11505,3186,say permit languished strickland director day ...,barely-true


In [160]:
filtered_df.to_csv(r'ShortStatements.csv', index = False)